In [5]:
pip install tensorflow openpyxl

DEPRECATION: Loading egg at /usr/local/Cellar/gpgme/1.22.0/lib/python3.11/site-packages/gpg-1.22.0-py3.11-macosx-13-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical

# Parameters
MAX_NUM_WORDS = 10000  # Max number of words in the tokenizer
MAX_SEQ_LENGTH = 200   # Max sequence length for padding
EMBEDDING_DIM = 128    # Dimension of word embeddings


In [9]:
# Load data from an Excel file (replace 'POC Based Requirements.xlsx' with your file)
data = pd.read_excel('POC Based Requirements.xlsx')

# Extract 'Requirement Description' and 'Requirements Bucket' columns as text data and labels
text_data = data['Requirement Description']
labels = data['Requirements Bucket']


In [10]:
# Tokenize the text data and create word indices
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(text_data)
word_index = tokenizer.word_index

# Convert text sequences to numerical sequences and pad them
sequences = tokenizer.texts_to_sequences(text_data)
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQ_LENGTH, padding='post', truncating='post')


In [11]:
# Convert category labels to one-hot vectors
labels_index = {category: idx for idx, category in enumerate(set(labels))}
label_count = len(labels_index)

labels = to_categorical(np.array(labels.map(labels_index)))


In [26]:
# Build the LSTM model
model = Sequential([
    Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQ_LENGTH),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(label_count, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [27]:
# Split your data into training, validation, and test sets (not included in the code)
# Use train_test_split or other methods to split your data into these sets

# Train the model
model.fit(padded_sequences, labels, epochs=10, validation_split=0.2, batch_size=40)


Epoch 1/10
2/2 [==============================] - 8s 2s/step - loss: 4.0435 - accuracy: 0.0147 - val_loss: 4.0385 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 1s 306ms/step - loss: 4.0384 - accuracy: 0.0441 - val_loss: 4.0440 - val_accuracy: 0.0000e+00
Epoch 3/10
2/2 [==============================] - 1s 259ms/step - loss: 4.0302 - accuracy: 0.0147 - val_loss: 4.0493 - val_accuracy: 0.0000e+00
Epoch 4/10
2/2 [==============================] - 1s 264ms/step - loss: 4.0336 - accuracy: 0.0147 - val_loss: 4.0553 - val_accuracy: 0.0000e+00
Epoch 5/10
2/2 [==============================] - 1s 264ms/step - loss: 4.0182 - accuracy: 0.0441 - val_loss: 4.0592 - val_accuracy: 0.0000e+00
Epoch 6/10
2/2 [==============================] - 1s 317ms/step - loss: 4.0067 - accuracy: 0.0588 - val_loss: 4.0659 - val_accuracy: 0.0000e+00
Epoch 7/10
2/2 [==============================] - 0s 244ms/step - loss: 3.9905 - accuracy: 0.0147 - val_loss: 4.0744 - val_accuracy: 0.0000

In [28]:
from sklearn.model_selection import train_test_split


# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(padded_sequences, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [29]:
# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {validation_loss:.4f}')
print(f'Validation Accuracy: {validation_accuracy:.4f}')


1/1 [==============================] - 0s 98ms/step - loss: 4.0532 - accuracy: 0.0000e+00
Validation Loss: 4.0532
Validation Accuracy: 0.0000


In [25]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


1/1 [==============================] - 0s 56ms/step - loss: 4.0294 - accuracy: 0.0000e+00
Test Loss: 4.0294
Test Accuracy: 0.0000
